# 1

扫描，记录上升下降的点，计算其中的面积。

出现上下变化时，记录左右边界；\
记录最高点，当前右边界如果不是目前的最高点，回退前面非最高点的记录。

（为了方便处理，为两端添加0。）（没用上）

没有完全通过。

In [83]:
from typing import List

class Solution:
    def trap(self, H: List[int]) -> int:
        l, r = 0, 0         # 索引
        pre = 0             # 上一个值
        max_h = 0           # 最高值
        is_max = False      # 是否是最高点的记录

        i = 0
        record = []         # [(l, r, 该点是否可能是小坑)] 小坑：该点高于最高点的话，就不是小坑。

        while i < len(H):
            is_max = False
            while i < len(H) and H[i] >= pre:     # 找左边界
                l = i
                if H[i] >= max_h:   # 记录目前的最高点
                    max_h = H[i]
                pre = H[i]
                i += 1
            while i < len(H) and H[i] <= pre:     # 找右边界1'
                pre = H[i]
                i += 1
            while i < len(H) and H[i] >= pre:     # 找右边界2'
                r = i
                if H[i] >= max_h:   # 遇到下一个最高点
                    is_max = True
                    max_h = H[i]
                while record and record[-1][-1] == False and record[-1][1] <= H[i]:    # 截止到is_max==True，如果当前高于前面记录的r时，回滚前面is_max==False的项
                    l, _, _ = record.pop()  # 回滚后，左边界更新
                pre = H[i]
                i += 1
            # print(l, r)       # debug
            if l < r:
                record.append((l, r, is_max))
            # print(record)     # debug
            l = r                                           # 保存新左边界

        ans = 0
        for l, r, _ in record:  # 计算雨水面积
            area = (r - l - 1) * min(H[l], H[r])  # w * h
            # print(f'l:{l} r:{r} w:{r - l - 1} h:{min(H[l], H[r])}', end=' ')
            for i in range(l+1, r):
                # print(f'(i:{i} -{H[i]})', end=' ')
                h = H[i] if H[i] <= min(H[l], H[r]) else min(H[l], H[r])  # 只减l, r最小的高度
                area -= h
            ans += area
            # print()
        return ans


if __name__ == '__main__':
    H_list = [
        [0,1,0,2,1,0,1,3,2,1,2,1],
        [1,0,1,0,2,1,0,1,4,4,1,2,1,0,3,1],
        [4,2,0,3,2,5],
        [5,4,1,2],
        [1,7,5],
        [8,2,8,9,0,1,7,7,9],
        [2,8,5,5,6,1,7,4,5],
        [6,4,2,0,3,2,0,3,1,4,5,3,2,7,5,3,0,1,2,1,3,4,6,8,1,3],
    ]

    for H in H_list:
        print(Solution().trap(H))

6
9
9
1
0
27
12
21


# 2

改变思路，扫描过去，记录最高点max_h；\
关键在于tmp记录前面出现，并不高于最高的点；\
接着每遇到一个点都记录gate[i]=H[i]，\
并往前查询这个tmp，将不高于当前点的都清除掉，认为没有闸门(gate[j]=0)。

In [124]:
from typing import List

class Solution:
    def trap(self, H: List[int]) -> int:
        max_h = 0
        gate = []
        tmp = []

        # 1.找到闸门
        for i in range(len(H)):
            if H[i] > 0:
                max_h = H[i]
                break

        for i in range(len(H)):
            for j in tmp:       # 核心：清除前面小于最高点的索引
                if H[j] <= H[i]:
                    gate[j] = 0

            tmp.append(i)
            gate.append(H[i])

            if H[i] >= max_h:
                max_h = H[i]
                tmp.clear()     # 核心：遇到更高的点，清除记录
        # print(gate)

        # 2.将闸门两端记录下来
        interval = []
        i = 0
        while i < len(gate) and gate[i] == 0:
            i += 1
        r = i
        while i < len(gate):
            l = r
            while i < len(gate) and gate[i] == 0:
                i += 1
            if i >= len(gate):
                break
            r = i
            interval.append([l, r])
            i += 1
        # print(interval)

        # 3.计算面积
        ans = 0
        for l, r in interval:
            if l == r or r-l == 1:  # 去除两端，以及紧贴在一起的
                continue
            area = (r - l - 1) * min(H[l], H[r])  # w * h
            for i in range(l+1, r):
                h = H[i] if H[i] <= min(H[l], H[r]) else min(H[l], H[r])  # 只减l, r最小的高度
                area -= h
            ans += area
        # print(ans)
        return ans

if __name__ == '__main__':
    H_list = [
        [0,1,0,2,1,0,1,3,2,1,2,1],
        [1,0,1,0,2,1,0,1,4,4,1,2,1,0,3,1],
        [0],
        [0,2,0],
        [4,2,0,3,2,5],
        [5,4,1,2],
        [1,7,5],
        [8,2,8,9,0,1,7,7,9],
        [2,8,5,5,6,1,7,4,5],
        [6,4,2,0,3,2,0,3,1,4,5,3,2,7,5,3,0,1,2,1,3,4,6,8,1,3],
    ]

    for H in H_list:
        print(Solution().trap(H))

6
14
0
0
9
1
0
27
12
83


执行用时 :956 ms, 在所有 Python3 提交中击败了5.01 %的用户

内存消耗 :17.4 MB, 在所有 Python3 提交中击败了5.06 %的用户

time: O(n^2)

space: O(n)

# 题解1

暴力法

计算当前点，左右的最高点，减去当前点的高度。 \
每个点都左右各查找一次。

time: O(n^2)

space: O(1)

# 题解2

动态规划 - 左右各扫描一遍

在暴力方法中，我们仅仅为了找到最大值每次都要向左和向右扫描一次。但是我们可以提前存储这个值。因此，可以通过动态编程解决。

通过以下图解，合并的重叠部分，很巧妙的就是积水部分。

![](https://pic.leetcode-cn.com/53ab7a66023039ed4dce42b709b4997d2ba0089077912d39a0b31d3572a55d0b-trapping_rain_water.png)

time: O(n)

space: O(n)

# 题解3

双指针

只要 right_max[i]>left_max[i]，积水高度将由 left_max 决定。

所以我们可以认为如果一端有更高的条形块（例如右端），积水的高度依赖于当前方向的高度（从左到右）。\
当我们发现另一侧（右侧）的条形块高度不是最高的，我们则开始从相反的方向遍历（从右到左）。

我们必须在遍历时维护 left_max 和 right_max ，但是我们现在可以使用两个指针交替进行，实现 1 次遍历即可完成。

time: O(n)

space: O(1)

（看着像11题，想过用双指针法，但没考虑到积水高度由一侧决定（关键），然后交替使用两个指针的方法。）